# 文本预处理

In [1]:
import torch
from torch import nn
import d2l.torch as d2l
import numpy as np
import re
import collections

读取数据集

In [2]:
# d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt', '090b5e7e70c295757f55df93cb0a180b9691891a')
path = '..\\data\\timemachine.txt'
def read_time_machine():
    with open(path, 'r') as f:
        lines = f.readlines() # 读出来是 3221 行的一个 list, 每一个元素都是txt里面的一行
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines] # 把除了 A-Z & a-z 之外的变成空格

lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')

# 文本总行数: 3221


看看这些行

In [3]:
print(lines[0])
print(lines[10])

the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


词元化

In [4]:
def tokenize(lines, token='word'):
    if token == 'word':
        return [line.split() for line in lines] # 按照空格 split
    elif token == 'char':
        return [list(line) for line in lines] # 直接转换成 list
tokens = tokenize(lines)
print(tokens[0])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']


词表

In [9]:
def count_corpus(tokens):
    # 统计标记的频率
    if len(tokens) == 0 or isinstance(tokens[0], list): # 空的就变成一个空列表, list 里是 list 就展开
        tokens = [token for line in tokens for token in line] # 将一个 list 里面的所有的 list 元素展开
        # 比如 [['a','b','c'],['e','6','i']] 变成 ['a','b','c','e','6','i']
    return collections.Counter(tokens) # 看一下每一个独一无二的 tokens 的个数

class Vocab:
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None): # reserved_tokens 这里还用不到, 就是句子开始&结束的token
        # min_freq 如果次数少于这个数, 就不计算, 把他当作 unk
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        counter = count_corpus(tokens) # 返回的是一个类似字典的东西, {'a':2} 表示a出现过两次
        self._token_freqs = sorted(counter.items(), key=lambda x:x[1], reverse=True) # x[1] 是出现的次数
        # items 是把字典的键值变成一个可迭代的tuple拿出来, ('a', 2) 这样
        # 按照从大到小的顺序进行排序, 因为 reverse 是 True
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token:idx for idx, token in enumerate(self.idx_to_token)} # 建立映射关系
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token) # 如果这个里面没有这个token
                self.token_to_idx[token] = len(self.idx_to_token) - 1 # 映射关系

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk) # 如果存在 tokens 则返回tokens的值, 否则返回unk
        return [self.__getitem__(token) for token in tokens]
    
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]
    
    @property
    def unk(self):
        return 0
    @property
    def token_freqs(self):
        return self._token_freqs

In [10]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [11]:
for i in [0,10]:
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]]) # 执行 __getitem__ 处

文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


整合所有功能

In [13]:
def load_corpus_time_machine(max_tokens=-1):
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0: # 截取最多多少个 token
        corpus = corpus[:max_tokens]
    return corpus, vocab 
    # corpus 是一长串的整数, 每一个整数对应的是一个 token, 也就是把原来的书变成数字
    # vocab 是一个字典

In [14]:
corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)